In [ ]:
!pip install insightface

In [ ]:
!pip install --upgrade onnxruntime


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install easydict


In [21]:
import argparse
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from create_face_bank import CreateFaceBank

class FaceIdentification:
    def __init__(self):
        try:
            self.app = FaceAnalysis(name="buffalo_s", providers=['CPUExecutionProvider'])
        except TypeError:
            self.app = FaceAnalysis(name="buffalo_s")
            print("⚠️ 'providers' argument not supported. Using default.")

        self.app.prepare(ctx_id=0, det_size=(640, 640))
        self.threshold = 25
        self.face_bank_path = "/content/drive/MyDrive/dataset/face_bank"

    def load_image(self, opt):
        self.input_image = cv2.imread(opt.image)
        self.results = self.app.get(self.input_image)

        if not self.results:
            print("⚠️ No face detected in the input image. Exiting...")
            return

    def load_face_bank(self):
        self.face_bank = np.load("/content/drive/MyDrive/pre_trained_model/face_bank.npy", allow_pickle=True)

    def update_face_bank(self):
        face_bank_creator = CreateFaceBank(self.face_bank_path)
        face_bank_creator.update(self.face_bank_path)

    def identification(self, opt):
        for result in self.results:
            cv2.rectangle(self.input_image, (int(result.bbox[0]), int(result.bbox[1])),
                          (int(result.bbox[2]), int(result.bbox[3])), (0, 255, 0), 4)

            for person in self.face_bank:
                distance = np.sqrt(np.sum((person["embedding"] - result["embedding"]) ** 2))
                if distance <= opt.threshold:
                    cv2.putText(self.input_image, person["name"],
                                (int(result.bbox[0]) - 50, int(result.bbox[1]) - 10),
                                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,
                                color=(0, 255, 0), thickness=2)
                    break
            else:
                cv2.putText(self.input_image, "Unknown",
                            (int(result.bbox[0]) - 50, int(result.bbox[1]) - 10),
                            fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,
                            color=(0, 0, 255), thickness=2)

        cv2.imwrite("result_image_4.jpg", self.input_image)
        print("✅ Identification completed. Result saved.")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--image', type=str, default="/content/images_4.jpg")
    parser.add_argument('--threshold', type=int, default=25)
    parser.add_argument('--update', action='store_true')

    # FIX: Ignore unrecognized arguments (solves Colab issue)
    opt, unknown = parser.parse_known_args()

    obj = FaceIdentification()
    obj.load_image(opt)

    if opt.update:
        obj.update_face_bank()

    obj.load_face_bank()
    obj.identification(opt)


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_s/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_s/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_s/det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_s/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_s/w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
✅ Identific